"## 내용 ##"에 해당하는 부분은 모두 수정해주어야 제대로 작동합니다.

###0. 라이브러리 임포트과 Config

In [ ]:
# 1. torch 임포트
import torch
# 2. 데이터셋
from torchvision import datasets ## 데이터셋을 위해 임포트해야 하는 것 ##
# 3. 데이터 변환
from torchvision.transforms import ToTensor, Lambda ## 이미지를 텐서로 만들기 위한 것 ##, ## 레이블을 원-핫 벡터로 만들기 위한 것 ##
# 4. 데이터로더
from torch.utils.data import DataLoader## 데이터로더 ##
# 5. 모델
import torch.nn as nn
# 6. 옵티마이저
import torch.optim as optim

In [ ]:
# 1. 학습률
learning_rate = 0.0001 ## 학습률 ##
# 2. 배치 사이즈
batch_size = 64 ## 배치 사이즈 ##
# 3. 에포크
epochs = 10 ## 에포크 ##

###1. 데이터셋과 데이터로더

In [ ]:
# 1. 학습 데이터셋과 검증 데이터셋
train_dataset = datasets.MNIST(
    root="data", ## 데이터셋을 저장할 경로 ##,
    train=True, ## 학습 데이터셋인지, 검증 데이터셋인지 ##,
    transform=ToTensor(), ## 이미지를 텐서로 만들기 위한 것 ##,
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download=True
)

val_dataset = datasets.MNIST(
    root="data", ## 데이터셋을 저장할 경로 ##,
    train=False, ## 학습 데이터셋인지, 검증 데이터셋인지 ##,
    transform=ToTensor(), ## 이미지를 텐서로 만들기 위한 것 ##,
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download=True
)

# 2. 학습 데이터로더와 검증 데이터로더
train_dataloader = DataLoader(
    dataset=train_dataset, ## 학습 데이터셋 ##,
    batch_size=64, ## 배치 사이즈 ##,
    shuffle=True ## 무작위 유무 ##
)

val_dataloader = DataLoader(
    dataset=val_dataset, ## 검증 데이터셋 ##,
    batch_size=64, ## 배치 사이즈 ##,
    shuffle=True ## 무작위 유무 ##
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



###2. 모델

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super().__init__() # 부모 클래스 init

        self.flatten = nn.Flatten() ## 데이터의 차원 바꾸어주기 ##
        ## 선형 회귀 모듈 ##
        self.linear = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, inputs):
        ## 무엇을 반환해야 할까? ##
        return self.linear(self.flatten(inputs))
        '''
        inputs = self.flatten(inputs)
        logits = self.linear(inputs)
        return logits
        '''

In [ ]:
model = LogisticRegression()

###3. 손실함수와 옵티마이저

In [ ]:
criterion = nn.CrossEntropyLoss() ## 10개의 클래스에 대한 분류에서는 어떤 손실 함수를 사용할까? ##

optimizer = optim.SGD(
    params=model.parameters(), ## 모델 파라미터 ##,
    lr=0.0001 ## 학습률 ## 1e-3
)

###4. 학습

In [ ]:
def train(model, dataloader, criterion, optimizer, device):

    model.train()

    total_loss = 0.0

    n_samples = len(dataloader.dataset)

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(inputs) ## 모델의 반환 값 ##

        # loss
        loss = criterion(outputs, labels) ## 손실 함수의 반환 값 ##
        total_loss += loss.item()

        # zero_grad
        ## 기울기를 0으로 만들기 ##
        optimizer.zero_grad()

        # backward
        ## 역전파를 통해 기울기 계산 ##
        loss.backward()

        # step
        ## 모델 파라미터 업데이트 ##
        optimizer.step()
    
    print(f"Train Loss: {total_loss / n_samples}")

###5. 검증

In [ ]:
def validation(model, dataloader, criterion, device):
    
    model.eval()

    n_samples = len(dataloader.dataset)

    total_loss = 0.0

    for inputs, labels in dataloader:
        inputs = inputs.to(device) ## 입력을 device로 보낸다. ##
        labels = labels.to(device) ## 레이블을 device로 보낸다. ##

        with torch.no_grad(): ## 검증 시에는 기울기를 계산할 필요가 없다. ##:
            # forward
            outputs = model(inputs) ## 모델 반환 값 ##

            # loss
            loss = criterion(outputs, labels).item() ## 손실 계산 ##
            total_loss += loss ## 손실 더하기 ##
        
    print(f"Validation Loss: {total_loss / n_samples}")

###6. 학습 실행

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" ## GPU 가속기를 사용할 수 있다면 ## else "cpu"
print(f"Using {device} device")

model = model.to(device) ## 모델을 device로 보낸다. ##

for inputs, labels in train_dataloader:
    print("dimension of inputs:", inputs.size())
    print("dimension of labels:", labels.size())
    break

for epoch in range(epochs):
    print(f"Epoch : {epoch + 1}")

    train(
        model=model,
        dataloader=train_dataloader,
        criterion=criterion,
        optimizer=optimizer,
        device=device
    )

    validation(
        model=model,
        dataloader=val_dataloader,
        criterion=criterion,
        device=device
    )

    print("==================================================")

Using cuda device
dimension of inputs: torch.Size([64, 1, 28, 28])
dimension of labels: torch.Size([64, 10])
Epoch : 1
Train Loss: 0.033408114953835805
Validation Loss: 0.033409003281593325
Epoch : 2
Train Loss: 0.033217777013778683
Validation Loss: 0.033209687662124635
Epoch : 3
Train Loss: 0.03301632546981176
Validation Loss: 0.03300049102306366
Epoch : 4
Train Loss: 0.032804333106676735
Validation Loss: 0.03277262716293335
Epoch : 5
Train Loss: 0.03257869292894999
Validation Loss: 0.03253592448234558
Epoch : 6
Train Loss: 0.03234115175207456
Validation Loss: 0.0322828706741333
Epoch : 7
Train Loss: 0.03208922613660495
Validation Loss: 0.032025319933891294
Epoch : 8
Train Loss: 0.0318241564532121
Validation Loss: 0.03174402573108673
Epoch : 9
Train Loss: 0.031543407688538236
Validation Loss: 0.0314529732465744
Epoch : 10
Train Loss: 0.031248419841130574
Validation Loss: 0.03113599418401718
